In [1]:
import os
import getpass # desc:这是为了安全地输入API密钥

# 设置环境变量以启用LangChain的跟踪功能
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("输入LangChain API Key: ")
# 设置项目名称以便在LangChain跟踪界面中区分不同项目
os.environ["LANGCHAIN_PROJECT"] = "llm-langchain-learn"

os.environ["OPENAI_API_KEY"] = getpass.getpass("请输入阿里云 API Key: ")

# 快速开始

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=os.environ["OPENAI_API_KEY"],
    model="qwen-plus"
)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [5]:
from langchain_core.messages import HumanMessage
# 使用刚创建的 model 进行对话
model.invoke(
    [HumanMessage(content="你好，AI！我是wlunan.")]
)

AIMessage(content='你好，wlunan！很高兴认识你～ 今天过得怎么样呀？希望你度过了愉快的一天。我随时准备好陪你聊天、帮你解决问题，或者就这样轻松愉快地闲聊一会儿。有什么想和我分享的吗？ 🌟', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 16, 'total_tokens': 68, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-dd68e763-e9c6-98da-9a7a-665384f200a8', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--8b4edb69-d097-4cab-ac7b-2f22502ee81c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 52, 'total_tokens': 68, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [6]:
# 但是ai没有状态,无法知道对话上下文
model.invoke([HumanMessage(content="我是")])

AIMessage(content='你好！看起来你可能想介绍自己。你可以告诉我你的名字或任何你想分享的信息吗？我很乐意认识你！ 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 9, 'total_tokens': 36, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-e725b91b-72e6-9444-b602-fdf5cdab28ce', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--ae94861e-e25f-4788-832f-a23a4dd8c957-0', usage_metadata={'input_tokens': 9, 'output_tokens': 27, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [7]:
# 为了了让AI知道上下文，我们需要把整个对话历史给它
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="你好，AI！我是wlunan."),
        AIMessage(content="你好，wlunan！很高兴认识你。"),
        HumanMessage(content="我是")
    ]
)

AIMessage(content='你好，wlunan！你刚才说“我是”，是不是想继续介绍一下自己呢？我在这里很乐意倾听和帮助哦。😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 36, 'total_tokens': 64, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-1d705532-7abc-9ee2-b878-f97845157d02', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--d7483775-51fc-4a35-bad8-78e4620e2db1-0', usage_metadata={'input_tokens': 36, 'output_tokens': 28, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

# 消息历史
我们可以使用消息历史类来包装我们的模型，使其具有状态。 这将跟踪模型的输入和输出，并将其存储在某个数据存储中。 未来的交互将加载这些消息，并将其作为输入的一部分传递给链。 

In [8]:
'''
BaseChatMessageHistory: 这是一个抽象基类，定义了聊天消息历史记录的接口。
InMemoryChatMessageHistory: 这是BaseChatMessageHistory的一个具体实现，使用内存来存储聊天消息历史记录。
RunnableWithMessageHistory: 这是一个可运行的类，扩展了基本的可运行类，添加了对聊天消息历史记录的支持。
'''
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 创建一个存储聊天历史记录的字典
store = {}
# 根据 session_id 获取对应的聊天历史记录
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
# 创建一个支持消息历史记录的可运行对象
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [10]:
# 我们现在需要创建一个 config，每次都传递给可运行的部分。这个配置包含的信息并不是直接作为输入的一部分，但仍然是有用的。
config = {"configurable": {"session_id": "abc2"}}
# 现在我们可以进行对话了，历史记录会被自动管理
response = with_message_history.invoke(
    [HumanMessage(content="你好,我是wlunan.")],
    config=config,
)

response.content

'你好，wlunan！欢迎来聊天～ 😊 很高兴认识你！今天过得怎么样呀？希望你度过了愉快的一天。如果有什么想聊的、想问的，或者需要帮忙的地方，尽管告诉我哦！ 🌟'

In [11]:
# 测试是否记住了历史
response = with_message_history.invoke(
    [HumanMessage(content="我的名字是")],
    config=config,
)

response.content

'你好，你刚才提到你的名字是 **wlunan** 呀！😊  \n如果名字有特别的读法或含义，也欢迎告诉我～  \n我们可以继续聊聊你想分享的事情！ 🌟'

这就是我们如何支持聊天机器人与多个用户进行对话的方式！

现在，我们所做的只是为模型添加了一个简单的持久化层。我们可以通过添加*提示词模板*来使其变得更加复杂和个性化。
# 提示词模版

提示词模板帮助将原始用户信息转换为大型语言模型可以处理的格式。在这种情况下，原始用户输入只是一个消息，我们将其传递给大型语言模型。现在让我们使其变得更复杂一些。首先，让我们添加一个带有一些自定义指令的系统消息（但仍然将消息作为输入）。接下来，我们将添加除了消息之外的更多输入。

首先，让我们添加一个系统消息。为此，我们将创建一个 ChatPromptTemplate。我们将利用 MessagesPlaceholder 来传递所有消息。

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [20]:
# 这稍微改变了输入类型 - 我们现在传递的是一个包含 messages 键的字典，其中包含一系列消息，而不是传递消息列表。
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

"Hi Bob! ٩(◕‿◕｡)۶ How's it going today?"

In [21]:
# 我们现在可以将其包装在与之前相同的消息历史对象中
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hello, I'm alice.")],
    config=config,
)
response.content

'Hello Alice! ٩(◕‿◕｡)۶ How can I assist you today?'

In [22]:
# 测试是否记住了历史
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Alice! 😊'

In [23]:
# 使用复杂的提示词
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Chinese"}
)

response.content

'你好，Bob！很高兴认识你！😊 今天过得怎么样？有什么我可以帮你的吗？'

In [26]:
# 现在让我们将这个更复杂的链封装在一个消息历史类中。这次，由于输入中有多个键，我们需要指定正确的键来保存聊天历史。
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Chinese"},
    config=config,
)

response.content

'嗨，Todd！很高兴认识你！有什么我可以帮你的吗？😊'

In [28]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Chinese"},
    config=config,
)

response.content

'你的名字是 Todd！😊'

# 管理对话历史
构建聊天机器人时，一个重要的概念是如何管理对话历史。如果不加以管理，消息列表将无限增长，并可能溢出大型语言模型的上下文窗口。因此，添加一个限制您传入消息大小的步骤是很重要的。

重要的是，您需要在提示模板之前但在从消息历史加载之前的消息之后执行此操作。

我们可以通过在提示前添加一个简单的步骤，适当地修改 messages 键，然后将该新链封装在消息历史类中来实现。

LangChain 提供了一些内置的助手来 管理消息列表。在这种情况下，我们将使用 trim_messages 助手来减少我们发送给模型的消息数量。修剪器允许我们指定希望保留的令牌数量，以及其他参数，例如是否希望始终保留系统消息以及是否允许部分消息：

In [43]:
from langchain_core.messages import SystemMessage, trim_messages

# 自定义近似 token 计数器，兼容 qwen-plus（官方未实现 get_num_tokens_from_messages）
def approx_token_count(msgs):
    # 粗略估算：约每 2 字符算 1 token，可按需调整
    return sum(len(getattr(m, "content", "")) for m in msgs) // 2

# trim_messages 参数说明：
# max_tokens: 最多保留多少个 token（约4个字符=1个token）
# strategy: "last" 表示保留最后的消息，"first" 表示保留最前面的
# token_counter: 用于计数 token 的模型或函数，这里用自定义近似计数器避免 qwen-plus 的 NotImplementedError
# include_system: 是否始终保留系统消息
# allow_partial: 是否允许部分消息（即使超过 token 限制）
# start_on: 从哪种消息类型开始计数（"human" 表示从人类消息开始）

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=approx_token_count,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

# 执行修剪操作，会返回不超过 45 个 token（近似）的消息列表
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [52]:
from operator import itemgetter # 
from langchain_core.runnables import RunnablePassthrough
# assign 让你在链的开头就对输入做局部变换/增加字段，
# 再把结果继续传下去；不用 assign 就不会有这步预处理。
chain = ( # "messages"是前面定义的输入键
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="我的名字是什么？")],
        "language": "Chinese",
    }
)
response.content
# 现在如果我们尝试询问模型我们的名字，它将不知道，因为我们修剪了聊天历史的那部分：

'抱歉，我不知道你的名字。你可以告诉我你叫什么吗？😊'

In [55]:
# 但是如果我们询问最近几条消息中的信息，它会记住：
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="我问了什么数学问题？")],
        "language": "Chinese",
    }
)
response.content

'你问了“2 + 2 等于多少？”，我回答是 4。😊'

In [56]:
# 包装在消息历史中
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "abc20"}}

In [58]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="我的名字是什么？")],
        "language": "Chinese"
    },
    config=config,
)
response.content

'我无法知道你的名字，因为这需要你告诉我。你可以告诉我你的名字，我会尊重并正确使用它。'

正如预期的那样，我们声明姓名的第一条消息已被删除。此外，聊天历史中现在有两条新消息（我们最新的问题和最新的回答）。这意味着以前可以在我们的对话历史中访问的更多信息现在不再可用！在这种情况下，我们最初的数学问题也已从历史中删除，因此模型不再知道它：

In [60]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="我问了什么数学问题？")],
        "language": "Chinese",
    },
    config=config,
)

response.content

'你还没有问过任何数学问题。如果你有任何数学问题，欢迎随时提问！'

# 流式处理
现在我们有了一个功能齐全的聊天机器人。然而，对于聊天机器人应用程序来说，一个非常重要的用户体验考虑是流式处理。大型语言模型有时可能需要一段时间才能响应，因此为了改善用户体验，大多数应用程序所做的一件事是随着每个令牌的生成流回。这样用户就可以看到进度。

实际上，这非常简单！

所有链都暴露一个.stream方法，使用消息历史的链也不例外。我们可以简单地使用该方法获取流式响应。


In [61]:
config = {"configurable": {"session_id": "abc15"}}

for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="你好,我是wlunan,给我讲一个笑话")],
        "language": "Chinese",
    },
    config=config,
):
    print(r.content, end="|")

|你好|，|wlunan|！很高兴|认识你～来|听个轻松的笑话|吧：

有一天，|小明去超市|买橙子，|他问老板：“|老板，你这橙子|甜吗？”  
老板|拍着胸脯说|：“当然甜！不|甜你砸我！”|  
小明拿起|一个橙子咬了一口|，结果特别酸，|立马把橙子朝|老板扔了过去。  
|老板大怒：“你|干嘛砸我？|！”  
小明理|直气壮地说：“我说|它不甜，|它就该砸你|啊！”

😄| 希望这个|笑话能让你笑一笑|！你也知道什么|好笑的段子吗|？分享一下呗|～||